In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
import ntpath
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score

def integer_to_onehot(data_integer, n_labels, start_at_zero=True):
    """
    Function to convert integer to onehot
    """
    if n_labels:
        data_onehot = np.zeros(shape=(data_integer.shape[0], n_labels))
    else:
        data_onehot = np.zeros(shape=(data_integer.shape[0], data_integer.max() + 1))
    for row in range(data_integer.shape[0]):
        
        integer = int(data_integer[row])
        if not start_at_zero:
            integer = integer - 1 
        data_onehot[row, integer] = 1
    return data_onehot

def extract_data(in_path, one_hot_labels_list, x_list, y_label_list, user_list, y_num_list,
                 activity_name_list):
    for dirname, _, filenames in os.walk(in_path):
        for filename in filenames:
            tmp_path = os.path.join(dirname, filename)
            #print(os.path.join(dirname, filename))
            df_data = pd.read_csv(tmp_path)
            user = ntpath.basename(tmp_path)[:-4]
            #return df_data
            #break
            df_data['y_num'] = 0
            #df_data.loc[(df_data['StartHesitation'] == 0) & (df_data['Turn'] == 0) & (df_data['Walking'] == 0), 'y_num'] = 1
            df_data.loc[df_data['StartHesitation'] == 1, 'y_num'] = 1
            df_data.loc[df_data['Turn'] == 1, 'y_num'] = 2
            df_data.loc[df_data['Walking'] == 1, 'y_num'] = 3

            df_data = add_identificator_of_change_in_variable(df_data, in_var='y_num')
            df_data['user'] = user
            #print(df_data['y_num'].unique())
            #plt.plot(df_data['state_period_number'])
            #plt.show()
            
            
            for label in df_data['state_period_number'].unique():
                # Select data for user
                df_pat_tmp = df_data.loc[df_data['state_period_number'] == label]
                
                if len(df_pat_tmp['y_num']) < 200:
                    continue
                x_list.append(np.asarray(df_pat_tmp[['AccV', 'AccML', 'AccAP']].iloc[0:200]))
                
                
                
                #return df_pat_tmp, df_data
                
                one_hot_labels_list.append(integer_to_onehot(df_pat_tmp['y_num'].iloc[0:200].to_numpy(), n_labels=4, start_at_zero=True))                
                user_list.append(user)
                y_num_list.append(df_pat_tmp['y_num'].iloc[0:200].to_numpy())    
                
                x_list.append(np.asarray(df_data[['AccV', 'AccML', 'AccAP']].iloc[df_data.Time.iloc[0] - 200:  df_data.Time.iloc[0]]))
                y_num_list.append(df_pat_tmp['y_num'].iloc[df_data.Time.iloc[0] - 200:  df_data.Time.iloc[0]].to_numpy())   
                one_hot_labels_list.append(integer_to_onehot(df_pat_tmp['y_num'].iloc[df_data.Time.iloc[0] - 200:  df_data.Time.iloc[0]].to_numpy(), n_labels=4, start_at_zero=True))
                user_list.append(user)
            
            
            

            del df_pat_tmp, df_data
    return one_hot_labels_list, x_list, y_label_list, user_list, y_num_list, activity_name_list


def add_identificator_of_change_in_variable(df_data, in_var):
    """

    """
    df_data.loc[df_data[in_var] < 1, 'state_number'] = 0
    df_data.loc[df_data[in_var] >= 1, 'state_number'] = 1
    #plt.plot(df_data[in_var])
    #plt.plot(df_data['state_number'])
    #plt.show()
    pa_df_patient = df_data[['state_number']].copy()
    df_data = df_data.drop(['state_number'], axis=1)

    pa_df_patient['change'] = pa_df_patient['state_number'].diff()


    df_tmp_2 = pa_df_patient.loc[pa_df_patient['state_number'] == 1, 'change'].copy()
    pa_df_patient['state_period_number'] = df_tmp_2.loc[~df_tmp_2.index.duplicated()].cumsum(axis=0)
    df_data = df_data.join(pa_df_patient, how='outer')
    # df_data['state_period_number'].plot()
    return df_data
"""
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
"""

'\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\n\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session\n'

In [2]:
 df, df_data = extract_data(
    train_dir1, one_hot_labels_list, x_list, y_label_list, user_list, y_num_list, activity_name_list)

NameError: name 'train_dir1' is not defined

In [ ]:
df.Time.iloc[0] - 200

In [ ]:
df_data.iloc[df.Time.iloc[0] - 200,  df.Time.iloc[0]]

In [ ]:
df

In [2]:
sampling_rate = 100 # Hz
time_required_second = 2  # s

number_of_points = sampling_rate * time_required_second

#time_

one_hot_labels_list, x_list, y_label_list, user_list, y_num_list = ([] for i in range(5))
activity_name_list = []
print(len(x_list))
train_dir1 = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog"
one_hot_labels_list, x_list, y_label_list, user_list, y_num_list, activity_name_list = extract_data(
    train_dir1, one_hot_labels_list, x_list, y_label_list, user_list, y_num_list, activity_name_list)

print(len(x_list))
train_dir2 = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog"
one_hot_labels_list, x_list, y_label_list, user_list, y_num_list, activity_name_list = extract_data(
    train_dir2, one_hot_labels_list, x_list, y_label_list, user_list, y_num_list, activity_name_list)

print(len(x_list))

0
1380
3132


In [ ]:
if False:
    for i in range(len(x_list)):
        plt.plot(x_list[i])
        plt.title(str(y_num_list[i]))
        #print(one_hot_labels_list[i])
        #print(y_num_list[i])
        #print(user_list[i])
        plt.show()

In [ ]:
train_set, test_set = train_test_split(range(len(x_list)))

In [3]:
dataset_train = {'x':  [x_list[i] for i in train_set],
                 'y': [y_num_list[i] for i in train_set],
                 'y_hot': [one_hot_labels_list[i] for i in train_set],
                 'user': [user_list[i] for i in train_set]
                  } 

dataset_test = {'x':  [x_list[i] for i in test_set],
                 'y': [y_num_list[i] for i in test_set],
                 'y_hot': [one_hot_labels_list[i] for i in test_set],
                 'user': [user_list[i] for i in test_set]
                  } 

NameError: name 'train_set' is not defined

In [4]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
dataset_train_y = [y[0] for y in dataset_train['y']]

In [ ]:
np.array(dataset_train_y)

In [ ]:
print("Before undersampling: ", Counter(dataset_train_y))

In [ ]:
dataset_train['x'][0].shape

In [ ]:
new_array = np.array([np.array([x[:, 0], x[:, 1], x[:, 2]]).reshape(600, ) for x in dataset_train['x']])
plt.plot(new_array[0])
plt.plot(dataset_train['x'][0])

In [ ]:
# define undersampling strategy
undersample = RandomUnderSampler(sampling_strategy='majority')


# fit and apply the transform
X_train_under, y_train_under = undersample.fit_resample(np.array(new_array), np.array(dataset_train_y))

In [ ]:
print("Before undersampling: ", Counter(y_train_under))

In [ ]:
X_train_under, y_train_under = undersample.fit_resample(X_train_under, y_train_under)

In [ ]:
X_train_under.shape

In [ ]:
print("Before undersampling: ", Counter(y_train_under))

In [ ]:
X_train_under[0][400:].shape

In [ ]:
y_train_under.shape[0]

In [ ]:
# integer_to_onehot(y_train_under, n_labels=3, start_at_zero=False)

In [ ]:
stacked_array = np.column_stack((X_train_under[0][0:200], X_train_under[0][200:400], X_train_under[0][400:]))
plt.plot(stacked_array)   

In [ ]:
X_train_under.shape

In [ ]:
dataset_train2 = {'x': X_train_under, #[np.column_stack((x[0:200], x[200:400], x[400:])) for x in X_train_under],
                 'y': y_train_under,
                 'y_hot': integer_to_onehot(y_train_under, n_labels=3, start_at_zero=False)
                }

In [ ]:
plt.hist([y for y in dataset_train2['y_hot']])

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
class MLPModel(tf.keras.Sequential):
    def __init__(self, input_shape, output_shape, nn=[40, 150], activations=['tanh', 'relu']):
        """
        This class is a MLP TensorFlow model
        :param input_shape: shape of the input layer
        :type input_shape: int
        :param output_shape: shape of the output layer
        :type output_shape: int
        :param nn: number of neurons in each layer
        :type nn: list
        :param activations: activation function in each layer
        :type activations: list
        """
        super().__init__()
        self.__class__.__name__ = "Sequential"

        for i in range(0, len(nn)):
            # add layer
            params_tmp = {'units': nn[i], 'activation': activations[i], 'kernel_initializer': "uniform"}
            if i == 0:
                params_tmp['input_shape'] = (None, input_shape)
            self.add(tf.keras.layers.Dense(**params_tmp))
        # add output layer
        self.add(tf.keras.layers.Dense(output_shape, activation='softmax'))

In [ ]:
class NNTensorFlow():
    """Model to wrap around tensorflow neural networks.
    """
    def __init__(self,
                 nn_key='MLP',
                 nn_params={},
                 n_epochs=10,
                 batch_size=100,
                 learning_rate=0.001,
                 cost_function="mean_squared_error",
                 metrics=['accuracy'],
                 verbose=True):
        """Instantiate the model.
        :param nn_params: Parameters to pass to the neural network.
        :type nn_params: dict
        :param n_epochs: The number of epochs.
        :type n_epochs: int
        :param batch_size: The batch size.
        :type batch_size: int
        :param learning_rate: The learning rate of the optimizer.
        :type learning_rate: float
        :param cost_function: Identifier of the cost function. Available losses in keras are described here:
        https://keras.io/api/losses/#available-losses
        :type cost_function: str
        :param verbose: Verbose to pass to model fit step.
        :type verbose: boolean
        """
        super().__init__()
        self.nn_params = nn_params
        if nn_key == 'MLP':
            self.nn_model = MLPModel(**nn_params)
        else:
            self.nn_model = AEModel(**nn_params)
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.cost_function = cost_function
        self.verbose = verbose
        self.metrics = metrics

    def fit(self, X, y, validation_data=None, continue_training=False):
        """
        Train the neural network.
        :param X: Training data.
        :type X: array-like or sparse matrix of shape (n_samples, n_features)
        :param y: Target values.
        :type y: array-like of shape (n_samples,) or (n_samples, n_targets)
        :param validation_data: (X, y) validation data
        :type validation_data: X is array-like or sparse matrix of shape (n_samples_val, n_features) and y is
                               Target validation values.
        :param continue_training: True to not re-instantiate the model each it is trained
        :type continue_training: bool
        :return: An instance of self.
        """

        if not continue_training:
            # Re-instantiate neural network in case of hyper-optimization
            self.nn_model = MLPModel(**self.nn_params)

        # Set optimizer
        optimizer_choice = tf.keras.optimizers.Adam(
            lr=self.learning_rate,
            beta_1=0.9,
            beta_2=0.999,
            amsgrad=False)

        # Compile
        # if self.cost_function in COST_FUNCTIONS:
        #     self.cost_function = COST_FUNCTIONS[self.cost_function]

        self.nn_model.compile(
            loss=self.cost_function,
            optimizer=optimizer_choice,
            metrics=self.metrics
        )

        # Train the model
        self.nn_model.fit(
            X, y,
            validation_data=validation_data,
            batch_size=self.batch_size,
            epochs=self.n_epochs,
            verbose=self.verbose)

        return self

    def predict(self, X):
        """Predict using the neural network.
        :param X: Samples.
        :type X: array_like or sparse matrix, shape (n_samples, n_features)
        :return: Predicted values.
        """
        return self.nn_model.predict(X)

    def save(self, model_path, model_name):
        """Saving the model.
        :param model_path: The path where to save it.
        :type model_path: string
        :param model_name: The name of the model.
        :type model_name: string
        """
        # pass
        save_keras_model(self.nn_model, model_path, model_name)

    def load(self, model_path, model_name):
        """Loading the model.
        :param model_path: The path where to save it.
        :type model_path: string
        :param model_name: The name of the model.
        :type model_name: string
        """

        self.nn_model = load_keras_model(model_path, model_name, custom_loss=self.cost_function)


        return self

In [ ]:
import os
import joblib
from sklearn.pipeline import Pipeline
#from utl.paths_utl import create_dir_path
#from etl.model_etl import save_default_model, load_default_model


class PipeModel(Pipeline):
    def __init__(self, steps, **params):
        """Instantiate the pipeline and the report.
        """
        super().__init__(steps, **params)
        self.report = {}

    def save(self, model_path, model_name):
        """Save each step of the pipeline.
        :param model_path: The path where to save it.
        :type model_path: string
        :param model_name: The name of the model.
        :type model_name: string
        """
        path_pipe = os.path.join(model_path, model_name)
        create_dir_path(path_pipe)

        for step_id, step in self.steps:

            if hasattr(step, 'save'):
                step.save(path_pipe, step_id)

            else:
                save_default_model(step, path_pipe, step_id)

    def load(self, model_path, model_name):
        """Load each step of the pipeline.
        :param model_path: The path where to save it.
        :type model_path: string
        :param model_name: The name of the model.
        :type model_name: string
        """
        path_pipe = os.path.join(model_path, model_name)

        new_steps = []
        for step_id, step in self.steps:

            if hasattr(step, 'load'):
                step_tmp = step.load(path_pipe, step_id)

            else:
                step_tmp = load_default_model(path_pipe, step_id)

            new_steps.append((step_id, step_tmp))

        self.steps = new_steps

        return self

    def get_pipeline_params(self):
        """Returns parameters for this pipeline that can be used to set parameters.
        :return: The parameters set for this pipeline.
        :rtype: dict
        """
        params = self.get_params()
        for step in params['steps']:
            params.pop(step[0])
        params.pop('steps')
        return params

    def get_report(self):
        """Return the reports from each step of the pipeline.
        """
        # Get report from each step
        for step_id, step in self.steps:
            if hasattr(step, 'report'):
                self.report[step_id] = step.report
            else:
                self.report[step_id] = None

        return self.report

In [ ]:
dataset_train2['x'].shape

In [ ]:
params2 = {
        'nn_key': 'MLP',
        'nn_params': {
            'input_shape': dataset_train2['x'].shape[1],
            'output_shape': 3,
            'nn': [500, 1000, 150],
            #'nn': [200, 150],
            'activations': ['tanh', 'tanh', 'relu']
            #'activations': ['tanh', 'relu']
        },
        'batch_size': 10,
        'cost_function': 'categorical_crossentropy',
        'learning_rate': 0.001,
        'n_epochs': 200,
    
        'metrics': ['accuracy']}

# Create pipeline
# model = Pipeline(steps=[('RobustScaler', StandardScaler()), ('NN', NNTensorFlow(**params2))])
pipe = PipeModel(steps=[('NN', NNTensorFlow(**params2))])



In [ ]:
X_val = np.asarray(dataset_train2['x'])
X = np.asarray(dataset_train2['x'])


In [ ]:
dataset_train2['x'].shape

In [ ]:
X.shape

In [ ]:
pipe['NN'].verbose = False

In [ ]:
y = np.array(dataset_train2['y_hot'])
#params_pipeline = {}
#params_pipeline[pipe.steps[-1][0] + "__validation_data"] = (X_val.reshape(X_val.shape[0], X_val.shape[2]), y)
pipe.fit(X, y)

In [ ]:
y_one_value = [y[0] - 1 for y in dataset_test['y']]

In [ ]:
def flatting_array_consecutive(in_array):
    return in_array.ravel('F')

def unflatting_array_consecutive(in_array, size=3):
    return in_array.reshape(size, -1).T

In [ ]:
x = dataset_test['x'][0]
plt.plot(x)

In [ ]:
y_pred = pipe.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['x']])))
y_pred.shape

In [ ]:
params_pipeline = {}
params_pipeline[pipe.steps[-1][0] + "__continue_training"] = True

In [ ]:
pipe['NN'].learning_rate

In [ ]:
pipe['NN'].learning_rate = 0.01
pipe.fit(X, y, **params_pipeline)
y_pred = pipe.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['x']])))
y_pred.shape
accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
pipe['NN'].learning_rate = 0.001
pipe.fit(X, y, **params_pipeline)
y_pred = pipe.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['x']])))
y_pred.shape
accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
plt.plot(y_one_value, 'k')
plt.plot(tf.argmax(y_pred, axis=1), 'r--')
plt.show()

In [ ]:
pipe['NN'].learning_rate = 0.0001
pipe.fit(X, y, **params_pipeline)
y_pred = pipe.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['x']])))
y_pred.shape
accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
plt.plot(y_one_value, 'k')
plt.plot(tf.argmax(y_pred, axis=1), 'r--')
plt.show()

In [ ]:
pipe['NN'].learning_rate = 0.01
pipe.fit(X, y, **params_pipeline)
y_pred = pipe.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['x']])))
y_pred.shape
accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
plt.plot(y_one_value, 'k')
plt.plot(tf.argmax(y_pred, axis=1), 'r--')
plt.show()

In [ ]:
pipe['NN'].learning_rate = 0.01
pipe.fit(X, y, **params_pipeline)
y_pred = pipe.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['x']])))
y_pred.shape
accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
tf.argmax(y_pred, axis=1)

In [ ]:
plt.plot(y_one_value, 'k')
plt.plot(tf.argmax(y_pred, axis=1), 'r--')
plt.show()

# Training using features

In [ ]:
dataset_train2['x'][0].shape

In [ ]:
stacked_array = unflatting_array_consecutive(dataset_train2['x'][0], size=3)

In [ ]:
plt.plot(dataset_train2['x'][0])
plt.plot(stacked_array)

## Proessing 3a

### Step 1: Filtering and calibration

In [ ]:
import numpy as np
from scipy import signal

# Constants
CUTOFF_FREQUENCY = 5.0  # Cut-off frequency for the low-pass filter

# Step 3: Pre-processing
# Apply low-pass filter to remove high-frequency noise
def apply_low_pass_filter(signal_data, cutoff):
    sampling_rate = 100.0  # Replace with actual sampling rate if known
    nyquist_frequency = 0.5 * sampling_rate
    normalized_cutoff = cutoff / nyquist_frequency
    b, a = signal.butter(4, normalized_cutoff, btype='low', analog=False)
    filtered_data = signal.filtfilt(b, a, signal_data, axis=0)
    return filtered_data

# Calibrate the data to remove bias or offset
def calibrate_data(signal_data):
    calibration_offsets = np.mean(signal_data, axis=0)
    calibrated_data = signal_data - calibration_offsets
    return calibrated_data

In [ ]:
# Apply pre-processing steps
preprocessed_data = stacked_array.copy()  # Make a copy of the data to avoid modifying the original
low_pass_filtered_data = apply_low_pass_filter(preprocessed_data, CUTOFF_FREQUENCY)
calibrated_data = calibrate_data(low_pass_filtered_data)
                                

In [ ]:
plt.plot(low_pass_filtered_data)
plt.plot(stacked_array)
plt.show()
plt.plot(calibrated_data)


## Step 2: Magnitud of acceleration

In [ ]:
def extract_features(in_data):
    """
    
    """
    # Example feature: Magnitude of the acceleration
    magnitudes = np.linalg.norm(in_data, axis=1)
    return magnitudes

In [ ]:
magnitud = extract_features(in_data=calibrated_data)

In [ ]:
plt.plot(calibrated_data)
plt.show()
plt.plot(magnitud)

## Step 3: Steps counts

In [ ]:
def detect_steps(in_features, in_window_size, in_threshold):
    """
    count steps
    """
    steps = 0    
    dict_patients = []
    for i in range(len(in_features) - in_window_size):
        feat = {}
        # Example algorithm: Threshold-based step detection
        window = in_features[i:i+in_window_size]
        feat['mean'] = np.nanmean(window)
        steps_window = 0
        for j in range(len(window)):
            if max(window) > in_threshold:
                steps_window += 1
        feat['steps_window'] = steps_window
            
        
        if max(window) > in_threshold:
            steps += 1
        dict_patients.append(feat)

    return dict_patients

In [ ]:
steps = detect_steps(in_features=magnitud, in_window_size=10, in_threshold=1.2)

In [ ]:
steps = {key: [feat[key] for feat in steps] for key in steps[0].keys()}

## Mean, std, rms, max, min, var, kurtosis, skewness, euclidean norm, l1 norm, fft, fft_energy, max_magnitude_fft

In [ ]:
import scipy.stats as stats
from scipy.fftpack import fft
def extract_features(in_features, in_window_size, in_stride=1):
    """
    extract mean
    """
    dict_patients = []
    for i in range(len(in_features) - in_window_size):
        feat = {}
        # Example algorithm: Threshold-based step detection
        window = in_features[i:i+in_window_size]
        feat['mean'] = np.nanmean(window, axis=0)
        feat['std'] = np.nanstd(window, axis=0)
        feat['rms'] = np.sqrt(np.mean(np.square(window), axis=0))
        feat['max'] = np.max(window, axis=0)
        feat['min'] = np.min(window, axis=0)
        feat['var'] = np.var(window, axis=0)
        feat['kurtosis'] =  stats.kurtosis(window, axis=0)
        feat['skew'] =  stats.skew(window, axis=0)
        
        feat['euclidean_norm'] = np.mean(np.linalg.norm(window, axis=1))
        
        feat['l1_norm'] = np.mean(np.linalg.norm(window, ord=1, axis=1))

        # Compute frequency domain metrics
        feat['fft'] = np.abs(fft(window, axis=0))
        feat['fft_energy'] = np.asarray(np.sum(feat['fft'] ** 2, axis=0) / len(feat['fft']))
        feat['max_magnitude_fft'] = np.max(feat['fft'], axis=0)
        feat['fft_x2'] = fft(window[:, 0])         
        dict_patients.append(feat)
    
    features = {key: [feat[key] for feat in dict_patients] for key in dict_patients[0].keys()}
    features['fft_energy'] = np.asarray(features['fft_energy'])
    features['max_magnitude_fft'] = np.asarray(features['max_magnitude_fft'])
    features['fft'] = np.asarray(features['fft'])
    return features
    

In [ ]:
features = extract_features(in_features=low_pass_filtered_data, in_window_size=40)
#features = {key: [feat[key] for feat in features] for key in features[0].keys()}
#features['fft_energy'] = np.asarray(features['fft_energy'])
#features['max_magnitude_fft'] = np.asarray(features['max_magnitude_fft'])
#features['fft'] = np.asarray(features['fft'])

In [ ]:
print(len(features['fft_energy']))
print(features['fft_energy'][0].shape)

In [ ]:
print(len(features['mean']))
print(features['mean'][0].shape)

In [ ]:
plt.plot(features['fft'][:, 2, :], 's')
plt.show()

In [ ]:
#features['max_magnitude_fft']

In [ ]:
plt.plot(low_pass_filtered_data, 'o')
plt.show()
plt.plot(features['std'], 'o')

in_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog'
for dirname, _, filenames in os.walk(in_path):
        for filename in filenames:
            tmp_path = os.path.join(dirname, filename)
            df_data = pd.read_csv(tmp_path)
            break
a3_data = df_data[['AccV', 'AccML', 'AccAP']].values

preprocessed_accelerometer_data = preprocess_data(data=a3_data)
magnitud = extract_features(in_data=preprocessed_accelerometer_data)
steps = detect_steps(in_features=magnitud, in_window_size=1, in_threshold=1.2)
plt.plot(a3_data)
plt.show()
plt.plot(preprocessed_accelerometer_data)
plt.show()
plt.plot(magnitud)
plt.show()
plt.plot(steps)



In [ ]:
dataset_features = []        
for x in dataset_train2['x']:
    x = unflatting_array_consecutive(x, size=3)
    #plt.plot(x)
    #plt.show()
    low_pass_filtered_data = apply_low_pass_filter(x, CUTOFF_FREQUENCY)
    #calibrated_data = calibrate_data(low_pass_filtered_data)
    features = extract_features(in_features=low_pass_filtered_data, in_window_size=40)
    dataset_features.append(features)
dataset_features = {key: [feat[key] for feat in dataset_features] for key in dataset_features[0].keys()}

    
    

In [ ]:
dataset_features.keys()

In [ ]:
dataset_train2.keys()

In [ ]:
dataset_train2.update(dataset_features)

In [ ]:
steps = detect_steps(in_features=magnitud, in_window_size=1, in_threshold=1.2)


In [ ]:
dataset_train2.keys()

In [ ]:
for key in ['mean', 'std', 'rms', 'max', 'min', 'var']:
    dataset_train2[key + 'flat'] = np.asarray(np.array([flatting_array_consecutive(np.asarray(x)) for x in dataset_train2[key]]))
    
#mean = flatting_array_consecutive(np.asarray(dataset_train2['mean'][0]))

In [ ]:
dataset_train2.keys()

In [ ]:
if False:
    for i in range(len(dataset_train2['meanflat'])):
        plt.plot(dataset_train2['meanflat'][i])
        plt.show()

In [ ]:
new_feat = []
for i in range(len(dataset_train2['meanflat'])):
    big_feat = []
    for keyi in ['meanflat', 'stdflat', 'rmsflat', 'maxflat', 'minflat', 'varflat']:
        #print(keyi)
        big_feat.append(dataset_train2[keyi][i])
        #plt.plot(dataset_train2[keyi][i])
        #plt.show()
    new_feat.append(np.asarray(np.concatenate(big_feat)))
dataset_train2['new_feat'] = new_feat
    #plt.plot(np.asarray(np.concatenate(big_feat)))
    #plt.show()
    #break

In [ ]:
#dataset_train2['new_feat']

In [ ]:
print(len(dataset_train2['new_feat']))
print(dataset_train2['new_feat'][0].shape)


In [ ]:
X_feat = np.asarray(dataset_train2['new_feat'])

In [ ]:
print(len(dataset_train2['new_feat']))
print(X_feat.shape)


In [ ]:
params3 = {
        'nn_key': 'MLP',
        'nn_params': {
            'input_shape': X_feat.shape[1],
            'output_shape': 3,
            'nn': [200, 150],
            'activations': ['tanh', 'relu']
        },
        'batch_size': 10,
        'cost_function': 'categorical_crossentropy',
        'learning_rate': 0.01,
        'n_epochs': 200,
    
        'metrics': ['accuracy']}

# Create pipeline
# model = Pipeline(steps=[('RobustScaler', StandardScaler()), ('NN', NNTensorFlow(**params2))])
pipe2 = PipeModel(steps=[('NN', NNTensorFlow(**params3))])


In [ ]:
pipe2['NN'].verbose = False

In [ ]:
y = np.array(dataset_train2['y_hot'])
#params_pipeline = {}
#params_pipeline[pipe.steps[-1][0] + "__validation_data"] = (X_val.reshape(X_val.shape[0], X_val.shape[2]), y)
pipe2.fit(X_feat, y)

In [ ]:
pipe2['NN'].learning_rate = 0.001
pipe2.fit(X_feat, y, **params_pipeline)


In [ ]:
pipe2['NN'].learning_rate = 0.0001
pipe2.fit(X_feat, y, **params_pipeline)

In [ ]:
dataset_test_features = []        
for x in dataset_test['x']:
    x = unflatting_array_consecutive(x, size=3)
    #plt.plot(x)
    #plt.show()
    low_pass_filtered_data = apply_low_pass_filter(x, CUTOFF_FREQUENCY)
    #calibrated_data = calibrate_data(low_pass_filtered_data)
    features = extract_features(in_features=low_pass_filtered_data, in_window_size=40)
    dataset_test_features.append(features)
dataset_test_features = {key: [feat[key] for feat in dataset_test_features] for key in dataset_test_features[0].keys()}

In [ ]:
dataset_test.update(dataset_test_features)

In [ ]:
for key in ['mean', 'std', 'rms', 'max', 'min', 'var']:
    dataset_test[key + 'flat'] = np.asarray(np.array([flatting_array_consecutive(np.asarray(x)) for x in dataset_test[key]]))
    

In [ ]:
new_feat = []
for i in range(len(dataset_test['meanflat'])):
    big_feat = []
    for keyi in ['meanflat', 'stdflat', 'rmsflat', 'maxflat', 'minflat', 'varflat']:
        # print(keyi)
        big_feat.append(dataset_test[keyi][i])
        #plt.plot(dataset_train2[keyi][i])
        #plt.show()
    new_feat.append(np.asarray(np.concatenate(big_feat)))
dataset_test['new_feat'] = new_feat
    #plt.plot(np.asarray(np.concatenate(big_feat)))
    #plt.show()
    #break

In [ ]:
y_pred = pipe2.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['new_feat']])))
y_pred.shape
accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
plt.plot(y_one_value, 'k')
plt.plot(tf.argmax(y_pred, axis=1), 'r--')
plt.show()

In [ ]:
pipe2['NN'].learning_rate = 0.0001
pipe2.fit(X_feat, y, **params_pipeline)

In [ ]:
y_pred = pipe2.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['new_feat']])))
y_pred.shape
accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
params_pipeline = {}
params_pipeline[pipe2.steps[-1][0] + "__continue_training"] = True

In [ ]:
pipe2['NN'].learning_rate = 0.0001
pipe2.fit(X_feat, y, **params_pipeline)
y_pred = pipe2.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['new_feat']])))
y_pred.shape
accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
plt.plot(y_one_value, 'k')
plt.plot(tf.argmax(y_pred, axis=1), 'r--')
plt.show()

In [ ]:
pipe2['NN'].learning_rate = 0.0001
pipe2.fit(X_feat, y, **params_pipeline)
y_pred = pipe2.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test['new_feat']])))
y_pred.shape
accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
plt.plot(y_one_value, 'k')
plt.plot(tf.argmax(y_pred, axis=1), 'r--')
plt.show()

# Test

In [ ]:
test_dir1 = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog"
in_path = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog"
for dirname, _, filenames in os.walk(in_path):
        for filename in filenames:
            tmp_path = os.path.join(dirname, filename)
            df_data = pd.read_csv(tmp_path)
            break
a3_data = df_data[['AccV', 'AccML', 'AccAP']].values

In [ ]:
a3_data.shape 

In [ ]:
# TODO: cut 200

In [ ]:
dataset_test_features = []        
in_window_size = 200
for a3 in [a3_data]:
    # In a signal
    for i in range(0, len(a3) - in_window_size, 1):
        # Example algorithm: Threshold-based step detection
        x = a3[i:i+in_window_size]  
        # plt.plot(x)
        # plt.show()
        low_pass_filtered_data = apply_low_pass_filter(x, CUTOFF_FREQUENCY)
        #calibrated_data = calibrate_data(low_pass_filtered_data)
        features = extract_features(in_features=low_pass_filtered_data, in_window_size=40)
        dataset_test_features.append(features)
dataset_test_features = {key: [feat[key] for feat in dataset_test_features] for key in dataset_test_features[0].keys()}

In [ ]:
len(dataset_test_features)

In [ ]:
plt.plot(dataset_test_features['mean'][2])

In [ ]:
dataset_test_features.keys()

In [ ]:
for key in ['mean', 'std', 'rms', 'max', 'min', 'var']:
    dataset_test_features[key + 'flat'] = np.asarray(np.array([flatting_array_consecutive(np.asarray(x)) for x in dataset_test_features[key]]))

In [ ]:
new_feat = []
for i in range(len(dataset_test_features['meanflat'])):
    big_feat = []
    for keyi in ['meanflat', 'stdflat', 'rmsflat', 'maxflat', 'minflat', 'varflat']:
        print(keyi)
        big_feat.append(dataset_test_features[keyi][i])
        #plt.plot(dataset_train2[keyi][i])
        #plt.show()
    new_feat.append(np.asarray(np.concatenate(big_feat)))
dataset_test_features['new_feat'] = new_feat
    #plt.plot(np.asarray(np.concatenate(big_feat)))
    #plt.show()
    #break

In [ ]:
dataset_test_features['new_feat'][1].shape 

In [ ]:
y_pred = pipe2.predict(np.asarray(np.array([flatting_array_consecutive(x) for x in dataset_test_features['new_feat']])))
y_pred.shape
#accuracy_score(integer_to_onehot(np.asarray(y_one_value), n_labels=3), integer_to_onehot(tf.argmax(y_pred, axis=1), n_labels=3))

In [ ]:
#plt.plot(y_one_value, 'k')
plt.plot(tf.argmax(y_pred, axis=1), 'r--')
plt.show()